In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Particionado") \
    .master("local[5]").getOrCreate()

In [2]:
df = spark.range(0,20)
df.rdd.getNumPartitions()

5

In [3]:
rdd1 = spark.sparkContext.parallelize((0,20), 10)
rdd1.getNumPartitions()

10

In [4]:
rddDesdeArchivo = spark \
    .sparkContext \
    .textFile("./files/deporte.csv", 10)

In [5]:
rddDesdeArchivo.getNumPartitions()

10

In [8]:
rddDesdeArchivo.saveAsTextFile("./partition_example/salidatexto")

In [13]:
!ls "./partition_example/salidatexto/"

_SUCCESS    part-00001	part-00003  part-00005	part-00007  part-00009
part-00000  part-00002	part-00004  part-00006	part-00008


SUCCESS es un archivo tipo log que si no hubo error siempre debe venir vacio

In [14]:
# Verificamos los archivos particionados
!head -n 5 ./partition_example/salidatexto/part-00004

29,Cycling
30,Diving
31,Canoeing
32,Tennis
33,Modern Pentathlon


Cargando los archivos particionados en un RDD

In [18]:
rdd = spark.sparkContext.wholeTextFiles('./partition_example/salidatexto/*')

In [21]:
rdd.take(2)

[('file:/home/rb/Platzi/Escuela de Data Science/22.- Curso Introductorio de Spark/partition_example/salidatexto/part-00009',
  '61,Croquet\n62,Jeu De Paume\n63,Roque\n64,Alpinism\n65,Basque Pelota\n66,Aeronautics\n'),
 ('file:/home/rb/Platzi/Escuela de Data Science/22.- Curso Introductorio de Spark/partition_example/salidatexto/part-00005',
  '36,Softball\n37,Archery\n38,Volleyball\n39,Synchronized Swimming\n40,Table Tennis\n41,Nordic Combined\n')]

In [22]:
# forma sencilla cargar
lista = rdd.mapValues(lambda x: x.split()).collect()

In [23]:
lista

[('file:/home/rb/Platzi/Escuela de Data Science/22.- Curso Introductorio de Spark/partition_example/salidatexto/part-00009',
  ['61,Croquet',
   '62,Jeu',
   'De',
   'Paume',
   '63,Roque',
   '64,Alpinism',
   '65,Basque',
   'Pelota',
   '66,Aeronautics']),
 ('file:/home/rb/Platzi/Escuela de Data Science/22.- Curso Introductorio de Spark/partition_example/salidatexto/part-00005',
  ['36,Softball',
   '37,Archery',
   '38,Volleyball',
   '39,Synchronized',
   'Swimming',
   '40,Table',
   'Tennis',
   '41,Nordic',
   'Combined']),
 ('file:/home/rb/Platzi/Escuela de Data Science/22.- Curso Introductorio de Spark/partition_example/salidatexto/part-00000',
  ['deporte_id,deporte',
   '1,Basketball',
   '2,Judo',
   '3,Football',
   '4,Tug-Of-War',
   '5,Speed',
   'Skating',
   '6,Cross',
   'Country',
   'Skiing']),
 ('file:/home/rb/Platzi/Escuela de Data Science/22.- Curso Introductorio de Spark/partition_example/salidatexto/part-00006',
  ['42,Baseball',
   '43,Rhythmic',
   'Gymnast

In [24]:
lista = [l[0] for l in lista]
lista.sort()

In [26]:
rdd2 = spark \
    .sparkContext \
    .textFile(','.join(lista), 10 ).map(lambda l: l.split(","))

In [27]:
rdd2.take(5)

[['deporte_id', 'deporte'],
 ['1', 'Basketball'],
 ['2', 'Judo'],
 ['3', 'Football'],
 ['4', 'Tug-Of-War']]